In [1]:
import os
import pandas as pd
import string
import re
from ast import literal_eval

#! pip install langdetect
from langdetect import detect

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

C:\Users\hp\AppData\Local\Temp\ipykernel_10552\1208028533.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
SPECIAL_CHARCTER_MAPPING = {
    'à': 'a', 'á': 'a', 'â': 'a', 'ã': 'a', 'ä': 'a', 'å': 'a',
    'æ': 'ae', 'ﬂ': 'fl', 'Đ': 'D', 'ﬁ': 'fi',
    'ç': 'c',
    'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e',
    'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i',
    'ð': 'd',
    'ñ': 'n',
    'ò': 'o', 'ó': 'o', 'ô': 'o', 'õ': 'o', 'ö': 'o', 'ø': 'o',
    'œ': 'oe',
    'ß': 'ss',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u',
    'ý': 'y', 'ÿ': 'y',
    'þ': 'th', 'Þ': 'Th',
    'ā': 'a', 'ă': 'a', 'ą': 'a', 'ȁ': 'a',
    'ć': 'c', 'ĉ': 'c', 'č': 'c', 'ċ': 'c',
    'đ': 'd', 'ď': 'd', 'ḑ': 'd',
    'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e', 'ę': 'e', 'ě': 'e', 'ĕ': 'e', 'ē': 'e',
    'ğ': 'g', 'ġ': 'g', 'ģ': 'g', 'ĝ': 'g',
    'ĥ': 'h', 'ħ': 'h', 'ĩ': 'i', 'ī': 'i', 'ĭ': 'i', 'į': 'i', 'ı': 'i',
    'ĵ': 'j', 'ǰ': 'j', 'ķ': 'k', 'ĺ': 'l', 'ļ': 'l', 'ľ': 'l', 'Ŀ': 'L', 'ŀ': 'l', 'ł': 'l',
    'ŉ': 'n', 'ń': 'n', 'ņ': 'n', 'ň': 'n', 'ŋ': 'n',
    'ŏ': 'o', 'ő': 'o', 'ō': 'o', 'ŏ': 'o', 'œ': 'oe',
    'ŕ': 'r', 'ř': 'r',
    'ś': 's', 'ŝ': 's', 'ş': 's', 'š': 's', 'ș': 's',
    'ť': 't', 'ţ': 't', 'ť': 't', 'ŧ': 't', 'ț': 't',
    'ũ': 'u', 'ū': 'u', 'ŭ': 'u', 'ů': 'u', 'ű': 'u', 'ų': 'u', 'ŵ': 'w', 'ŷ': 'y', 'ÿ': 'y',
    'ź': 'z', 'ž': 'z', 'ż': 'z',
    'ă': 'a', 'ĕ': 'e', 'ĭ': 'i', 'ŏ': 'o', 'ŭ': 'u',
    'ā': 'a', 'ē': 'e', 'ī': 'i', 'ō': 'o', 'ū': 'u',
    'ǎ': 'a', 'ě': 'e', 'ǐ': 'i', 'ǒ': 'o', 'ǔ': 'u',
    'ȁ': 'a', 'ȅ': 'e', 'ȉ': 'i', 'ȍ': 'o', 'ȕ': 'u',
    'ą': 'a', 'ę': 'e', 'į': 'i', 'ų': 'u',
    'ć': 'c', 'ĉ': 'c', 'č': 'c', 'ċ': 'c',
    'đ': 'd', 'ď': 'd', 'ḑ': 'd',
    'ĝ': 'g', 'ğ': 'g', 'ġ': 'g', 'ģ': 'g',
    'ĥ': 'h', 'ħ': 'h',
    'ĵ': 'j', 'ǰ': 'j',
    'ķ': 'k', 'ĸ': 'k',
    'ĺ': 'l', 'ļ': 'l', 'ľ': 'l', 'ŀ': 'l', 'ł': 'l',
    'ḿ': 'm', 'ṁ': 'm', 'ṃ': 'm',
    'ń': 'n', 'ŉ': 'n', 'ŋ': 'n',
    'ŕ': 'r', 'ŗ': 'r', 'ř': 'r',
    'ś': 's', 'ŝ': 's', 'ş': 's', 'š': 's', 'ș': 's',
    'ţ': 't', 'ť': 't', 'ŧ': 't',
    'ũ': 'u', 'ū': 'u', 'ŭ': 'u', 'ů': 'u', 'ű': 'u', 'ų': 'u',
    'ŵ': 'w',
    'ỳ': 'y', 'ý': 'y', 'ÿ': 'y',
    'ź': 'z', 'ż': 'z', 'ž': 'z',
    'ạ': 'a', 'ả': 'a', 'ấ': 'a', 'ầ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ậ': 'a', 'ắ': 'a', 'ằ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ặ': 'a',
    'ắ': 'a', 'ằ': 'a', 'ẳ': 'a', 'ẵ': 'a',
    'ặ': 'a', 'ẹ': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ế': 'e', 'ề': 'e', 'ể': 'e', 'ễ': 'e', 'ệ': 'e', 'ỉ': 'i', 'ị': 'i', 'ọ': 'o',
    'ỏ': 'o', 'ố': 'o', 'ồ': 'o', 'ổ': 'o', 'ỗ': 'o', 'ộ': 'o', 'ớ': 'o', 'ờ': 'o', 'ở': 'o', 'ỡ': 'o', 'ợ': 'o', 'ụ': 'u',
    'ủ': 'u', 'ứ': 'u', 'ừ': 'u', 'ử': 'u', 'ữ': 'u', 'ự': 'u', 'ỳ': 'y', 'ỵ': 'y', 'ỷ': 'y', 'ỹ': 'y',
}

In [8]:
def clean_poem(poem) :
  poem = literal_eval(poem)

  n = len(poem)
  empty_lines_count = 0
  punctuation_count = 0
  for i in range(n) :
    poem[i] = poem[i].replace('\u200a', ' ').replace('\u2009', ' ')
    if poem[i] in ['___', '_____']: poem[i] = ''
    punctuation_count += sum(1 for char in poem[i] if char in string.punctuation)
    poem[i] = re.sub(r'[^\w\s]', ' ', poem[i]).strip()
    poem[i] = re.sub('\d+', '', poem[i]).strip()
    poem[i] = re.sub(r'\s+', ' ', poem[i]).strip()
    
    translation_table = str.maketrans(SPECIAL_CHARCTER_MAPPING)    
    poem[i] = poem[i].translate(translation_table)
    poem[i] = re.sub('[^a-zA-Z]', ' ', poem[i]).strip()
    
    if poem[i]=='' :
      empty_lines_count += 1
      continue

  poem = [verse.strip() for verse in poem if verse.strip()]

  # The following step is CRITICAL !!!!
  try :
    pom = [verse for verse in poem if detect(verse)=='en']
  except :
    print(poem)

  return punctuation_count, empty_lines_count, pom

<>:12: SyntaxWarning: invalid escape sequence '\d'
<>:12: SyntaxWarning: invalid escape sequence '\d'
C:\Users\hp\AppData\Local\Temp\ipykernel_10552\2936059097.py:12: SyntaxWarning: invalid escape sequence '\d'
  poem[i] = re.sub('\d+', '', poem[i]).strip()


In [9]:
data_dir = '9_class_data'
os.makedirs(data_dir, exist_ok=True)

for csv_file in os.listdir('.'):
    if csv_file.endswith('.csv') and csv_file!='data.csv':

        category = os.path.splitext(csv_file)[0]

        category_dir = os.path.join(data_dir, category)
        os.makedirs(category_dir, exist_ok=True)

        df = pd.read_csv(csv_file)
        if 'Unnamed: 0' in df.columns :
            df.drop('Unnamed: 0', axis=1, inplace=True)
        df['Category'] = [category]*len(df)
        #df['Author'].fillna('Others', inplace=True)
        #df['Punctuation_Count'], df['empty_lines_count'], df['Poem'] = zip(*df['Poem'].apply(clean_poem))

        for index, poem in enumerate(df['Poem']):
            poem_file_path = os.path.join(category_dir, f'{category}_{index}.txt')
            with open(poem_file_path, 'w') as f:
                f.write('\n'.join(poem))
            df.at[index, 'Poem'] = poem_file_path
        
        df.to_csv('new_' + csv_file, index=False)

print("Process completed successfully.") #44

Process completed successfully.


In [10]:
dfs = []

# Iterate over each CSV file in the current directory
for csv_file in os.listdir('.'):
    if csv_file.endswith('.csv'):
        # Read the CSV file and append DataFrame to dfs list
        df = pd.read_csv(csv_file)
        dfs.append(df)

# Concatenate all DataFrames in dfs list
data = pd.concat(dfs, ignore_index=True)

# Write the concatenated DataFrame to data.csv
data.to_csv('data.csv', index=False)

print("Concatenation completed successfully.")

Concatenation completed successfully.


In [11]:
len(data) # 2648 --> 3368

3638